# A first finite DMRG Example

This notebook shows the basic interface for DMRG.
It initalized the Heisenberg model and runs DMRG to find the ground state.

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
np.set_printoptions(precision=5, suppress=True, linewidth=100)
plt.rcParams['figure.dpi'] = 150

In [ ]:
import tenpy
import tenpy.linalg.np_conserved as npc
from tenpy.algorithms import dmrg
from tenpy.networks.mps import MPS
from tenpy.models.spins import SpinChain

from tenpy.networks.site import SpinHalfSite
from tenpy.models.lattice import Chain

tenpy.tools.misc.setup_logging(to_stdout="INFO")

In [ ]:
# Define the length of the chain and initalize the model:

L = 32
model_params = {
    'Jz': 1.,
    'L': L,
    'bc_MPS': 'finite',    
}

M = SpinChain(model_params)

In [ ]:
# This inializes an MPS with Sz_tot = 0:

psi = MPS.from_lat_product_state(M.lat, [['up'],['down']])

In [ ]:
# This cell does the same as the one above, but gives access to the MPS.
# One can use it the chain the initial MPS in some detail:

site = SpinHalfSite(conserve='Sz')  # predefined charges and Sp,Sm,Sz operators
p_leg = site.leg
chinfo = p_leg.chinfo

# create product state MPS
state = ["up", "down"] * (L // 2) + ["up"] * (L % 2)  # Neel state
#state = ["up"] * (L // 2) + ["down"] * (L - L // 2) 

# Switch one spin:
state[0] = "down"
print("state = ", state)

psi = MPS.from_product_state(M.lat.mps_sites(), state, M.lat.bc_MPS)


In [ ]:
# This runs DMRG with some standard parameters:

dmrg_params = {
    'mixer': None,  # setting this to True helps to escape local minima
    'max_E_err': 1.e-10,
    'trunc_params': {
        'chi_max': 100,
        'svd_min': 1.e-10,
    },
    'verbose': True,
    'combine': True
}
eng = dmrg.TwoSiteDMRGEngine(psi, M, dmrg_params) 
E, psi = eng.run() # the main work; modifies psi in place

## Expectation Values

In [ ]:
# onsite expectation values

Z = psi.expectation_value("Sz")
x = np.arange(psi.L)

plt.figure()
plt.plot(x, Z, label="Z")
plt.xlabel("site")
plt.ylabel("onsite expectation value")
plt.legend()
plt.show()

In [ ]:
# correlation functions
Z = psi.expectation_value("Sz")

i0 = psi.L // 4  # for fixed `i`
j = np.arange(i0 + 1, psi.L)

ZZ = psi.term_correlation_function_right([("Sz", 0)], [("Sz", 0)], i_L=i0, j_R=j)
ZZ_disc = ZZ - Z[i0] * Z[j]

dx = j - i0
plt.figure()
#plt.plot(dx, XX_disc, label="X X")
plt.plot(dx, ZZ_disc, label="Z Z")
plt.xlabel(r"distance $|i-j|$")
plt.ylabel(r"disconnected correlations $\langle A_i A_j\rangle - \langle A_i \rangle\langle A_j\rangle$")
plt.legend()
#plt.loglog()
plt.show()

In [ ]:
print(j)